# Getting started with Starling (ST)


In [ ]:
%pip install https://github.com/camlab-bioml/starling/archive/main.zip
%pip install lightning_lite

import anndata as ad
import pandas as pd
import torch
from lightning_lite import seed_everything
from pytorch_lightning.callbacks import EarlyStopping  # ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

from starling import starling, utility

     / 8.9 MB 15.4 MB/s 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.6/159.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.6/774.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 55.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32

In [ ]:
torch.cuda.is_available()

True

## Setting seed for everything


In [ ]:
# pl.utilities.seed.seed_everything(10, workers=True)
seed_everything(10, workers=True)

INFO:lightning_lite.utilities.seed:Global seed set to 10


10

## Loading annData objects


The example below runs Kmeans with 10 clusters read from "sample_input.h5ad" object.


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
anndata = ad.read_h5ad("data\train\cell_data.h5ad") #/content/drive/MyDrive/cell_data.h5ad

In [ ]:
anndata

AnnData object with n_obs × n_vars = 236791 × 40
    obs: 'image', 'sample_id', 'ObjectNumber', 'Pos_X', 'Pos_Y', 'area', 'major_axis_length', 'minor_axis_length', 'eccentricity', 'width_px', 'height_px', 'acquisition_id', 'SlideId', 'Study', 'Box.Description', 'Position', 'SampleId', 'Indication', 'BatchId', 'SubBatchId', 'ROI', 'ROIonSlide', 'includeImage', 'flag_no_cells', 'flag_no_ROI', 'flag_total_area', 'flag_percent_covered', 'small_cell', 'celltypes', 'flag_tumor', 'PD1_pos', 'Ki67_pos', 'cleavedPARP_pos', 'GrzB_pos', 'tumor_patches', 'distToCells', 'CD20_patches', 'Batch', 'cell_labels'
    var: 'channel', 'use_channel', 'marker'
    layers: 'counts', 'exprs'

In [ ]:
print(anndata.shape)
print(anndata.obs_names)
print(anndata.var_names)
print(anndata.var_keys())
print(anndata.varm_keys())
print(anndata.obsm)
print(anndata.layers)
print(anndata.layers['exprs'].shape)
print(anndata.layers['counts'].shape)
print(anndata.layers.keys())
print(anndata.X)

anndata.X = anndata.layers['exprs'] # FIX!
print(anndata.X)

(236791, 40)
Index(['IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_1',
       'IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_3',
       'IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_5',
       'IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_7',
       'IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_8',
       'IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_12',
       'IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_21',
       'IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_23',
       'IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_28',
       'IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_30',
       ...
       'IMMUcan_Batch20220908_S-220729-00002_002.tiff_2707',
       'IMMUcan_Batch20220908_S-220729-00002_002.tiff_2708',
       'IMMUcan_Batch20220908_S-220729-00002_002.tiff_2709',
       'IMMUcan_Batch20220908_S-220729

In [ ]:
adata = utility.init_clustering("KM", anndata, k=10)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


- Users might want to arcsinh protein expressions in \*.h5ad (for example, 'sample_input.h5ad').
- The utility.py provides an easy setup of GMM, KM (Kmeans) or PG (PhenoGraph).
- Default settings are applied to each method.
- k can be omitted when PG is used.


## Setting initializations


The example below uses defualt parameter settings based on benchmarking results (more details in manuscript).


In [ ]:
st = starling.ST(adata)

A list of parameters are shown:

- adata: annDATA object of the sample
- dist_option (default: 'T'): T for Student-T (df=2) and N for Normal (Gaussian)
- the proportion of anticipated segmentation error free cells (default: 0.6)
- model_cell_size (default: 'Y'): Y for incoporating cell size in the model and N otherwise
- cell_size_col_name (default: 'area'): area is the column name in anndata.obs dataframe
- model_zplane_overlap (default: 'Y'): Y for modeling z-plane overlap when cell size is modelled and N otherwise
  Note: if the user sets model_cell_size = 'N', then model_zplane_overlap is ignored
- model_regularizer (default: 1): Regularizier term impose on synthetic doublet loss (BCE)
- learning_rate (default: 1e-3): The learning rate of ADAM optimizer for STARLING

Equivalent as the above example:
st = starling.ST(adata, 'T', 'Y', 'area', 'Y', 1, 1e-3)


## Setting trainning log


Once training starts, a new directory 'log' will created.


In [ ]:
## log training results via tensorboard
log_tb = TensorBoardLogger(save_dir="log")

One could view the training information via tensorboard. Please refer to torch lightning (https://lightning.ai/docs/pytorch/stable/api_references.html#profiler) for other possible loggers.


## Setting early stopping criterion


In [ ]:
## set early stopping criterion
cb_early_stopping = EarlyStopping(monitor="train_loss", mode="min", verbose=False)

Training loss is monitored.


## Training Starling


In [ ]:
## train ST
st.train_and_fit(
    callbacks=[cb_early_stopping],
    logger=[log_tb],
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name | Type | Params
------------------------------
------------------------------
0         Trainable params
0         Non-trainable params
0         Total params
0.000     Total estimated model params size (MB)
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:563: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get Dat

Training: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/starling/utility.py:500: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2985.)
  prob_data_given_gamma_d1.T + log_delta[1] - prob_data
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


## Appending STARLING results to annData object


In [ ]:
## retrive starling results
result = st.result()

## The following information can be retrived from annData object:

- st.adata.varm['init_exp_centroids'] -- initial expression cluster centroids (P x C matrix)
- st.adata.varm['st_exp_centroids'] -- ST expression cluster centroids (P x C matrix)
- st.adata.uns['init_cell_size_centroids'] -- initial cell size centroids if STARLING models cell size
- st.adata.uns['st_cell_size_centroids'] -- initial & ST cell size centroids if ST models cell size
- st.adata.obsm['assignment_prob_matrix'] -- cell assignment probability (N x C maxtrix)
- st.adata.obsm['gamma_prob_matrix'] -- gamma probabilitiy of two cells (N x C x C maxtrix)
- st.adata.obs['doublet'] -- doublet indicator
- st.adata.obs['doublet_prob'] -- doublet probabilities
- st.adata.obs['init_label'] -- initial assignments
- st.adata.obs['st_label'] -- ST assignments
- st.adata.obs['max_assign_prob'] -- ST max probabilites of assignments
  - N: # of cells; C: # of clusters; P: # of proteins


## Saving the model


In [ ]:
## st object can be saved
torch.save(st, "model.pt")

model.pt will be saved in the same directory as this notebook.


## Showing STARLING results


In [ ]:
display(result)

AnnData object with n_obs × n_vars = 236791 × 40
    obs: 'image', 'sample_id', 'ObjectNumber', 'Pos_X', 'Pos_Y', 'area', 'major_axis_length', 'minor_axis_length', 'eccentricity', 'width_px', 'height_px', 'acquisition_id', 'SlideId', 'Study', 'Box.Description', 'Position', 'SampleId', 'Indication', 'BatchId', 'SubBatchId', 'ROI', 'ROIonSlide', 'includeImage', 'flag_no_cells', 'flag_no_ROI', 'flag_total_area', 'flag_percent_covered', 'small_cell', 'celltypes', 'flag_tumor', 'PD1_pos', 'Ki67_pos', 'cleavedPARP_pos', 'GrzB_pos', 'tumor_patches', 'distToCells', 'CD20_patches', 'Batch', 'cell_labels', 'init_label', 'st_label', 'doublet_prob', 'doublet', 'max_assign_prob'
    var: 'channel', 'use_channel', 'marker'
    uns: 'init_cell_size_centroids', 'init_cell_size_variances', 'st_cell_size_centroids'
    obsm: 'assignment_prob_matrix', 'gamma_assignment_prob_matrix'
    varm: 'init_exp_centroids', 'init_exp_variances', 'st_exp_centroids'
    layers: 'counts', 'exprs'

One could easily perform further analysis such as co-occurance, enrichment analysis and etc.


In [ ]:
adata.obs

,image,sample_id,ObjectNumber,Pos_X,Pos_Y,area,major_axis_length,minor_axis_length,eccentricity,width_px,...,tumor_patches,distToCells,CD20_patches,Batch,cell_labels,init_label,st_label,doublet_prob,doublet,max_assign_prob
IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_1,IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01...,IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01...,1.0,300.846154,0.692308,13.0,6.094800,2.780135,0.889904,600.0,...,1,8.773580,,Batch20191023,MacCD163,5,5,0.307670,0,0.692314
IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_3,IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01...,IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01...,3.0,26.982143,0.928571,56.0,21.520654,3.368407,0.987675,600.0,...,0,72.247393,,Batch20191023,Mural,3,3,0.294797,0,0.702338
IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_5,IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01...,IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01...,5.0,309.083333,0.750000,12.0,5.294329,2.862220,0.841267,600.0,...,1,16.982199,,Batch20191023,DC,5,5,0.084695,0,0.915300
IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_7,IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01...,IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01...,7.0,431.916667,0.750000,12.0,5.294329,2.862220,0.841267,600.0,...,1,-8.314676,,Batch20191023,Tumor,5,9,0.999764,1,0.000196
IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_8,IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01...,IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01...,8.0,116.931034,1.206897,29.0,9.216670,4.112503,0.894932,600.0,...,1,-15.358007,,Batch20191023,Tumor,3,8,0.958385,1,0.038224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
IMMUcan_Batch20220908_S-220729-00002_002.tiff_2713,IMMUcan_Batch20220908_S-220729-00002_002.tiff,IMMUcan_Batch20220908_S-220729-00002_002,2713.0,596.548387,596.709677,31.0,6.857501,5.700162,0.555928,600.0,...,0,85.376518,,Batch20220908,Mural,3,3,0.942602,1,0.032497
IMMUcan_Batch20220908_S-220729-00002_002.tiff_2715,IMMUcan_Batch20220908_S-220729-00002_002.tiff,IMMUcan_Batch20220908_S-220729-00002_002,2715.0,180.300000,597.400000,20.0,6.484816,3.840203,0.805803,600.0,...,1,10.318477,,Batch20220908,Mural,3,3,0.074980,0,0.924740
IMMUcan_Batch20220908_S-220729-00002_002.tiff_2721,IMMUcan_Batch20220908_S-220729-00002_002.tiff,IMMUcan_Batch20220908_S-220729-00002_002,2721.0,48.370370,598.111111,27.0,10.732613,3.134663,0.956397,600.0,...,1,14.074760,,Batch20220908,CD8,9,9,0.489163,0,0.498525
IMMUcan_Batch20220908_S-220729-00002_002.tiff_2722,IMMUcan_Batch20220908_S-220729-00002_002.tiff,IMMUcan_Batch20220908_S-220729-00002_002,2722.0,207.969697,598.060606,33.0,12.864691,3.228974,0.967988,600.0,...,0,26.662288,,Batch20220908,Mural,3,3,0.277201,0,0.721467


In [ ]:
result.obs

,image,sample_id,ObjectNumber,Pos_X,Pos_Y,area,major_axis_length,minor_axis_length,eccentricity,width_px,...,tumor_patches,distToCells,CD20_patches,Batch,cell_labels,init_label,st_label,doublet_prob,doublet,max_assign_prob
IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_1,IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01...,IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01...,1.0,300.846154,0.692308,13.0,6.094800,2.780135,0.889904,600.0,...,1,8.773580,,Batch20191023,MacCD163,5,5,0.307670,0,0.692314
IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_3,IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01...,IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01...,3.0,26.982143,0.928571,56.0,21.520654,3.368407,0.987675,600.0,...,0,72.247393,,Batch20191023,Mural,3,3,0.294797,0,0.702338
IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_5,IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01...,IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01...,5.0,309.083333,0.750000,12.0,5.294329,2.862220,0.841267,600.0,...,1,16.982199,,Batch20191023,DC,5,5,0.084695,0,0.915300
IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_7,IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01...,IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01...,7.0,431.916667,0.750000,12.0,5.294329,2.862220,0.841267,600.0,...,1,-8.314676,,Batch20191023,Tumor,5,9,0.999764,1,0.000196
IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_8,IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01...,IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01...,8.0,116.931034,1.206897,29.0,9.216670,4.112503,0.894932,600.0,...,1,-15.358007,,Batch20191023,Tumor,3,8,0.958385,1,0.038224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
IMMUcan_Batch20220908_S-220729-00002_002.tiff_2713,IMMUcan_Batch20220908_S-220729-00002_002.tiff,IMMUcan_Batch20220908_S-220729-00002_002,2713.0,596.548387,596.709677,31.0,6.857501,5.700162,0.555928,600.0,...,0,85.376518,,Batch20220908,Mural,3,3,0.942602,1,0.032497
IMMUcan_Batch20220908_S-220729-00002_002.tiff_2715,IMMUcan_Batch20220908_S-220729-00002_002.tiff,IMMUcan_Batch20220908_S-220729-00002_002,2715.0,180.300000,597.400000,20.0,6.484816,3.840203,0.805803,600.0,...,1,10.318477,,Batch20220908,Mural,3,3,0.074980,0,0.924740
IMMUcan_Batch20220908_S-220729-00002_002.tiff_2721,IMMUcan_Batch20220908_S-220729-00002_002.tiff,IMMUcan_Batch20220908_S-220729-00002_002,2721.0,48.370370,598.111111,27.0,10.732613,3.134663,0.956397,600.0,...,1,14.074760,,Batch20220908,CD8,9,9,0.489163,0,0.498525
IMMUcan_Batch20220908_S-220729-00002_002.tiff_2722,IMMUcan_Batch20220908_S-220729-00002_002.tiff,IMMUcan_Batch20220908_S-220729-00002_002,2722.0,207.969697,598.060606,33.0,12.864691,3.228974,0.967988,600.0,...,0,26.662288,,Batch20220908,Mural,3,3,0.277201,0,0.721467


Starling provides doublet probabilities and cell assignment if it were a singlet for each cell.


## Showing initial expression centriods:


In [ ]:
## initial expression centriods (p x c) matrix
pd.DataFrame(result.varm["init_exp_centroids"], index=result.var_names)

,0,1,2,3,4,5,6,7,8,9
0,0.182287,0.247703,0.364653,0.175833,0.187019,0.249961,0.215287,0.171826,1.102367,0.189744
1,2.442940,3.373201,3.655138,2.369185,2.088048,2.465966,3.183247,2.098939,3.325023,2.588923
2,0.271024,0.368260,0.722134,1.009973,0.253049,0.526537,0.408264,0.163406,0.529890,0.471192
3,0.400646,0.803550,2.155967,0.417647,0.460627,1.835485,0.595478,0.265208,0.853055,0.948789
4,0.488598,0.642263,1.578283,0.417673,0.155841,0.453393,0.571709,0.234827,0.542955,0.500537
5,3.582649,1.773323,3.870111,1.235916,1.611744,3.583422,1.917782,0.696858,2.077758,2.736263
6,1.484366,0.943213,2.018156,0.650459,0.396608,0.769688,0.840776,0.382303,0.954505,1.245242
7,0.637775,0.941030,1.570171,0.521612,1.474706,0.901271,0.772101,1.673750,4.630700,0.741276
8,2.851509,0.998044,1.596058,0.511688,0.300339,0.601371,0.832569,0.445428,0.761463,0.766572
9,0.488696,0.388181,1.643557,0.333587,0.290623,2.055905,0.281961,0.149020,0.706454,0.821908


There are 10 centroids since we set Kmeans (KM) as k = 10 earlier.


## Showing Starling expression centriods:


In [ ]:
## starling expression centriods (p x c) matrix
pd.DataFrame(result.varm["st_exp_centroids"], index=result.var_names)

,0,1,2,3,4,5,6,7,8,9
0,0.159696,0.335304,0.295207,0.159250,0.144499,0.190584,0.175062,0.144954,0.541748,0.162294
1,2.429575,3.979391,3.647428,2.324338,1.988204,2.264828,3.052928,2.069797,3.391776,2.522532
2,0.079683,0.177124,0.448332,0.737563,0.113301,0.234712,0.174798,0.052150,0.567999,0.264641
3,0.268492,1.143314,2.345245,0.310479,0.159991,1.877465,0.363031,0.227918,0.542334,0.899677
4,0.396603,0.967539,1.370221,0.178175,0.098151,0.290967,0.395829,0.220922,0.695085,0.309618
5,3.601013,2.024531,4.021672,1.251094,0.793518,3.670582,1.127517,0.447215,1.684782,2.837458
6,1.634540,1.348114,1.993974,0.482241,0.229482,0.599409,0.681092,0.365776,1.139716,1.134852
7,0.299956,0.748124,0.812305,0.229053,1.332277,0.601999,0.397235,1.274690,3.074246,0.398310
8,2.926410,1.305023,1.361369,0.314047,0.184171,0.413794,0.679942,0.408972,0.888991,0.455551
9,0.188161,0.391098,1.747860,0.208177,0.084322,2.033178,0.183629,0.119286,0.444237,0.630430


From here one could easily annotate cluster centriods to cell type.


## Showing Assignment Distributions:


In [ ]:
## assignment distributions (n x c maxtrix)
pd.DataFrame(result.obsm["assignment_prob_matrix"], index=result.obs.index)

,0,1,2,3,4,5,6,7,8,9
IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_1,3.434608e-16,1.277297e-24,1.658661e-22,2.805119e-06,1.969041e-18,6.923135e-01,1.446472e-15,2.075221e-17,1.726391e-09,1.365450e-05
IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_3,2.892067e-13,1.950499e-14,1.333046e-20,7.023376e-01,1.318666e-18,6.820942e-10,4.339510e-05,1.873683e-10,2.820779e-03,7.960766e-07
IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_5,2.130106e-15,6.628046e-13,1.305426e-07,2.764341e-15,4.428631e-31,9.152996e-01,3.425492e-17,3.151642e-26,7.428256e-10,5.682892e-06
IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_7,5.931229e-13,7.163635e-09,8.880445e-10,1.385379e-15,1.547233e-34,2.937688e-05,5.524966e-12,1.981426e-23,1.086485e-05,1.961111e-04
IMMUcan_batch20191023_10032145-THOR-VAR-TIS-01-IMC-01_002.tiff_8,7.908969e-13,1.248286e-10,7.374103e-14,6.380908e-04,1.758281e-18,1.276397e-04,2.182376e-03,6.154912e-11,3.822390e-02,4.432047e-04
...,...,...,...,...,...,...,...,...,...,...
IMMUcan_Batch20220908_S-220729-00002_002.tiff_2713,3.931836e-12,6.710713e-16,6.819566e-17,3.249737e-02,3.081557e-15,4.490567e-05,2.182340e-13,7.636922e-14,2.460969e-02,2.462828e-04
IMMUcan_Batch20220908_S-220729-00002_002.tiff_2715,1.027244e-21,4.114542e-36,2.079754e-37,9.247405e-01,2.790277e-04,1.260272e-17,7.179071e-21,1.494797e-13,5.579380e-19,6.431683e-16
IMMUcan_Batch20220908_S-220729-00002_002.tiff_2721,3.219343e-10,3.965365e-20,2.585818e-12,7.106793e-12,4.193653e-32,1.231194e-02,3.476479e-19,2.201129e-24,3.544698e-07,4.985252e-01
IMMUcan_Batch20220908_S-220729-00002_002.tiff_2722,1.015064e-19,9.172164e-32,4.587079e-34,7.214671e-01,1.332108e-03,1.811031e-16,1.581951e-16,2.104927e-11,1.300518e-15,1.339720e-13


Currently, we assign a cell label based on the maximum probability among all possible clusters. However, there could be mislabeled because maximum and second highest probabilies can be very close that the user might be interested.
